In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import datetime as dt
import ipywidgets as widgets
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 6)

movies = pd.read_csv("../data/cleaned_movie_data.csv")
movies.head()

,adult,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,release_date,title,...,Science Fiction,TV Movie,Thriller,War,Western,overview_len,month,day,day_of_week,noisy_month
0,0,/bOGkgRGdhrBYJSLpXaxhXVstddV.jpg,299536,en,Avengers: Infinity War,as the avengers and their allies have continue...,153.811,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,2018-04-25,Avengers: Infinity War,...,0,0,0,0,0,490,4,25,Wed,3.834536
1,0,/5zfVNTrkhMu673zma6qhFzG01ig.jpg,300668,en,Annihilation,a biologist signs up for a dangerous secret e...,29.516,/d3qcpfNwbAMCNqWDHzPQsUYiUgS.jpg,2018-02-22,Annihilation,...,1,0,0,0,0,116,2,22,Thu,2.108673
2,0,/zjG95oDnBcFKMPgBEmmuNVOMC90.jpg,299782,en,The Other Side of the Wind,surrounded by fans and skeptics grizzled dire...,6.820,/kFky1paYEfHxfCYByEc9g7gn6Zk.jpg,2018-11-02,The Other Side of the Wind,...,0,0,0,0,0,188,11,2,Fri,11.172691
3,0,/q9hnJ9SzwcF30seRtXEzLd5l1gw.jpg,351044,en,Welcome to Marwen,when a devastating attack shatters mark hoganc...,61.973,/o45VIAUYDcVCGuzd43l8Sr5Dfti.jpg,2018-12-21,Welcome to Marwen,...,0,0,0,0,0,413,12,21,Fri,12.084792
4,0,/AmO8I38bkHwKhgxPNrd6djBQyPU.jpg,361292,en,Suspiria,a darkness swirls at the center of a world ren...,41.461,/dzWTnkert9EoiPWldWJ15dnfAFl.jpg,2018-10-11,Suspiria,...,0,0,1,0,0,242,10,11,Thu,9.890125


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
# split into training and test sets
train, test = train_test_split(movies)

### Processing the Data

Before we can fit a model, we first need to process the data in the `train` dataframe so that we end up with only numerical feature columns. To this end, we define the helper function `cols` which selects a list of columns out of a dataframe and `process_data` which processes the dataframe and extract the list of features that we want to use. The `process_data` function first drops rows with `nan` values in the `overview_len` column and then one-hot encodes the `day_of_week` variable, before selecting out the desired feature columns `features`. Finally, we also define our loss function `rmse` which computes the root-mean-squared error.

In [4]:
def process_data(data, features):
    """
    Processes input data for linear regression model, returns feature 
    matrix `X` and array of actual values `y`
    
    Args:
        data     - dataframe with the data
        features - columns of features to choose
    """
    data = data.dropna(subset=["overview_len"])
    
    data = pd.get_dummies(data,
                          prefix = "wkdy",
                          columns = ["day_of_week"],
                          drop_first = True
                         )
        
    X = data[features]
    y = data["vote_average"]
    return X, y

def rmse(y, y_hat):
    """
    Compute the root-mean-squared error between actual values `y` 
    and fitted/predicted values `y_hat`
    
    Args:
        y     - array of actual values
        y_hat - array of fitted values
    """
    return np.sqrt(np.mean((y - y_hat)**2))

To test our functions above, we process the data in our training and test sets using three random features in `sample_features`.

In [6]:
sample_features = ["vote_count", "adult", "day"]
sample_X_train, sample_y_train = process_data(train, sample_features)
sample_X_test, sample_y_test = process_data(test, sample_features)

In order to facilitate easy running of the linear regression model, we define the function `run_lin_reg` below which runs the `sklearn` linear regression pipeline. It starts by initializing a new `LinearRegression` instance, then fits the training data and then predicts the training and testing data. If its `verbose` argument is `True`, it prints out the training and testing RMSEs, and it returns the testing RMSE.

In [7]:
def run_lin_reg(X_train, y_train, X_test, y_test, verbose=True):
    """
    Runs linear regression using `X_train`, `y_train` as training
    set and `X_test`, `y_test` as test set. Returns test RMSE. If
    `verbose`, will print out training and test RMSEs.
    
    Args:
        X_train - dataframe or matrix of feature values (training 
                  set)
        y_train - array of actual values for training set
        X_test  - dataframe or matrix of feature values (test set)
        y_test  - array of actual values for test set
        verbose - boolean to indicate whether to print out results
    """
    lr = lm.LinearRegression()
    lr.fit(X_train, y_train)
    y_fitted = lr.predict(X_train)
    y_pred = lr.predict(X_test)

    training_rmse = rmse(y_train, y_fitted)
    testing_rmse = rmse(y_test, y_pred)

    if verbose:
        print("Training RMSE: {:.3f}\nTesting RMSE: {:.3f}".format(
            training_rmse, testing_rmse
        ))

    return testing_rmse
    
run_lin_reg(sample_X_train, sample_y_train, sample_X_test, sample_y_test)

Training RMSE: 2.131
Testing RMSE: 2.059


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


2.0585210347959917

This is a high RMSE given that the ratings are on a 10 point scale, but these are only sample features, and we will attempt to choose better ones below using $k$-fold cross validation. 

### 4-Fold Cross Validation

To that end, we define the function `run_kfold` which takes in training data and features and runs 4-fold cross validation on the training set. It returns the average cross validation RMSE, which will then be used to select features in the next cell.

In [8]:
def run_kfold(train_data, features):
    """
    Runs k-fold cv and returns RMSE
    
    Args:
        train_data - dataframe of matrix, training set
        features   - list of features to select from training
                     set
    """
    X, y = process_data(train_data, features)
    
    kf = KFold(n_splits = 4)
    cv_rmses = []
    for train_idx, valid_idx in kf.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        cv_rmse = run_lin_reg(X_train, y_train, X_valid, y_valid, False)
        cv_rmses += [cv_rmse]
        
    return np.mean(cv_rmses)

run_kfold(train, sample_features)

2.1329752661561057

To choose features, we implement the function `choose_features`, which runs through a list of lists of features and returns the set of features with the lowest CV RMSE. To test the function, we create another feature set `more_sample_features` to run against `sample_features`.

In [9]:
more_sample_features = ["vote_count", "adult", "Fantasy", "overview_len"]
sample_features_rmse = run_kfold(train, sample_features)
more_sample_features_rmse = run_kfold(train, more_sample_features)
print("{} RMSE: {:.5f}".format(sample_features, sample_features_rmse))
print("{} RMSE: {:.5f}".format(more_sample_features, more_sample_features_rmse))

better_rmse = (sample_features, more_sample_features)[sample_features_rmse > more_sample_features_rmse]

def choose_features(train_data, feature_sets):
    """
    Function to run k-fold CV on a list of sets of features and
    return the features with best mean RMSE.
    
    Args:
        train_data   - dataframe of matrix, training set
        feature_sets - list of sets of features (nested list)
    """
    best_error, best_features = np.inf, []
    for features in feature_sets:
        cv_error = run_kfold(train_data, features)
        if cv_error < best_error:
            best_error = cv_error
            best_features = features
    
    return best_features

better_sample_features = choose_features(train, [sample_features, more_sample_features])
print("`choose_features` chose: {}".format(better_sample_features))
print("Function chose correct set: {}".format(better_sample_features == better_rmse))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  import sys


['vote_count', 'adult', 'day'] RMSE: 2.13298
['vote_count', 'adult', 'Fantasy', 'overview_len'] RMSE: 2.14247
`choose_features` chose: ['vote_count', 'adult', 'day']
Function chose correct set: True


Now, in order to choose the best features, we import the list of genres to create a list of all possible features `possible_features`.

In [10]:
genres = pd.read_csv("../data/tmdb_genres.csv")
genres["id"] = genres["id"].astype(str)
genre_dict = {d["id"] : d["name"] for d in genres.to_dict(orient="records")}
possible_features = list(genre_dict.values())
possible_features.extend([
    "adult", "vote_count", "overview_len", "month", "day", "wkdy_Mon", "wkdy_Sat", "wkdy_Sun",
    "wkdy_Thu", "wkdy_Tue", "wkdy_Wed"
])

Because running through all of the possible combinations of features would take too long (if we only consider combinations of 15 features, there are 155117520 possibilities), we define a function `stochastic_choose_features` which uses a stochastic algorithm to choose the best set of features. The algorithm is fairly simple:

1. Choose a random number of features $n \in \{1, 2, ..., x-1, x \}$, where $x$ is the length of `possible_features` (for now, 30)
2. Choose a simple random sample of $n$ features from the set of possible features
3. Compare this set with the better set of features in the previous iteration (if iteration 1, a randomly chosen set of features)
4. Set `best_features` to the better set of features
5. Rinse and repeat until `maxiter` iterations have been done

In [11]:
def stochastic_choose_features(train_data, features, maxiter=1000):
    """
    Uses a stochastic algorithm to find the best set of features by randomly
    selecting the number of features to use in each iteration and then
    running KFold CV to determine which feature set is best. Returns the set
    of features that had lowest average CV RMSE
    
    Args:
        train_data - dataframe of matrix, training set
        features   - list of all possible features to consider
        maxiter    - number of iterations to go through before returning best
                     features; default = 1000
    """
    n = np.random.choice(range(1, len(features)+1))
    best_features = np.random.choice(features, size=n, replace=False)
    for _ in range(maxiter):
        n = np.random.choice(range(1, len(features)+1))
        chosen_features = np.random.choice(features, size=n, replace=False)
        best_features = choose_features(train_data, [chosen_features, best_features])
        
    return list(best_features)

stochastic_features = stochastic_choose_features(train, possible_features)
print("Features chosen: ", stochastic_features)
print("Features KFold CV RMSE: ", run_kfold(train, stochastic_features))

Features chosen:  ['TV Movie', 'vote_count', 'Fantasy', 'Horror', 'day', 'Romance', 'month', 'Comedy', 'Thriller', 'wkdy_Mon', 'Crime', 'Drama', 'wkdy_Sat', 'Documentary', 'War', 'Family', 'adult', 'wkdy_Tue']
Features KFold CV RMSE:  2.0558763370665973


However, the algorithm in the above function takes a while to run since each iteration requires the previous one to complete before the next one can begin, and also because it doesn't take advantage of all of the cores on the computer.

### Parallelizing Computation

In order to make up for the shortcomings of `stochastic_choose_features`, we create a pipeline below using the `ray` package to parallelize the choosing of the best set of features using a similar stochastic algorithm. The only difference here is that we will first initialize all `maxiter` feature sets and then apply a function to each pair of them using tree reduction until there is only one set of features left.

To begin this process, we import `ray` below and initialize the workers.

In [12]:
import ray

ray.init(ignore_reinit_error=True);

2019-04-30 16:22:39,976	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-30_16-22-39_2928/logs.
2019-04-30 16:22:40,087	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:29706 to respond...
2019-04-30 16:22:40,247	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:17498 to respond...
2019-04-30 16:22:40,251	INFO services.py:804 -- Starting Redis shard with 3.44 GB max memory.
2019-04-30 16:22:40,287	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-30_16-22-39_2928/logs.
2019-04-30 16:22:40,290	INFO services.py:1427 -- Starting the Plasma object store with 5.15 GB memory using /tmp.


Using the `@ray.remote` decorator, we first define the function that will use the tree reduction algorithm on the list of items. It works by applying the reducer function to pairs of elements in the list until there is only 1 item left, which it returns. The comparator will be the function that chooses which set of features is better given two sets.

In [13]:
@ray.remote
def reduce_iterations(f, train_data, items):
    """
    
    """
    while len(items) > 1:
        reduced = [f.remote(train_data, items[2*i], items[2*i+1]) for i in range(len(items)//2)]
        items = (reduced, reduced + [items[-1]])[len(items) % 2 == 1]
    return items

Next, we adapt the `choose_features` function from above to allow it to be used as a `ray` remote function. The new function is called `one_stochastic_iteration` because it is essentially doing the work of a single iteration of our stochastic feature choice algorithm.

In [14]:
@ray.remote
def one_stochastic_iteration(train_data, features_1, features_2):
    return list(choose_features(train_data, [features_1, features_2]))

Now we define a function to help choose features. This function corresponds to steps 1 and 2 from the algorithm above.

In [15]:
def choose_random_features(features):
    n = np.random.choice(range(1, len(features)+1))
    chosen_features = np.random.choice(features, size=n, replace=False)
    return list(chosen_features)

Finally, we put the entire pipeline into a function `feature_pipeline` which we will be able to reuse later. It creates all `maxiter` feature sets before using the reducer function to apply tree reduction to the list of feature sets. It returns a ray `ObjectID`. We then call the function using the `train` dataframe, the `possible_features` list, and the three functions we defined above.

In [16]:
def feature_pipeline(one_iteration, chooser, reducer, train_data, features, maxiter=1000):
    sets = []
    for _ in range(maxiter):
        sets += [chooser(features)]
    result_id = reducer.remote(one_iteration, train_data, sets)
    return result_id

features_id = feature_pipeline(
    one_stochastic_iteration, 
    choose_random_features,
    reduce_iterations, 
    train, 
    possible_features
)

The function `feature_pipeline`, because of how it is designed, returns an `ObjectID` which points to a list containing another `ObjectID` which points to the actual result. For this reason, we need to call `ray.get` twice in order to extract the list of features.

In [17]:
stochastic_features = ray.get(ray.get(features_id))[0]
stochastic_features

(pid=2981) /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
(pid=2981)   linalg.lstsq(X, y)
(pid=2980) /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
(pid=2980)   linalg.lstsq(X, y)
(pid=2987) /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result o

['Action',
 'Thriller',
 'Documentary',
 'Western',
 'TV Movie',
 'adult',
 'Comedy',
 'wkdy_Sat',
 'day',
 'vote_count',
 'wkdy_Mon',
 'Romance',
 'Horror',
 'Mystery',
 'Drama',
 'War']

Now, to test the set of features chosen by the `stochastic_choose_features` function, we run the linear regression on the test set and get the testing RMSE.

In [18]:
X_train, y_train = process_data(train, stochastic_features)
X_test, y_test = process_data(test, stochastic_features)

run_lin_reg(X_train, y_train, X_test, y_test)

Training RMSE: 2.039
Testing RMSE: 1.936


1.9363291585515696

The RMSE is high given that the ratings are on a 10-point scale, so now we revisit feature engineering to see if we can come up with a better model.

## Feature Engineering II

In this section of the notebook, we attempt to build a better model by doing some more EDA and feature engineering.

### Words as Features

To begin our consideration of new features, we consider the presence of words in plots. Below, we write a function to add a column to a dataframe that indicates whether or not a word is present in that movies overview (`1` for yes, `0` for no).

In [19]:
def add_word_in_plot(df, words):
    """
    Adds column to dataframe indicating whether or not a word appears in
    that movie's overview
    
    Args:
        df    - dataframe to consider, must contain `overview` column
        words - list of words to add as columns
    """
    df = df.copy()
    df = df.dropna(subset=["overview"])
    for word in words:
        df[word] = df["overview"].str.contains(word).astype(int)
    return df

add_word_in_plot(movies, ["love"]).head()

,id,title,overview,genres,release_date,adult,vote_count,vote_average,Action,Adventure,...,Science Fiction,TV Movie,Thriller,War,Western,overview_len,month,day,day_of_week,love
0,299536,Avengers: Infinity War,as the avengers and their allies have continue...,"['Adventure', 'Action', 'Fantasy']",2018-04-25,False,12490,8.3,1,1,...,0,0,0,0,0,490.0,4,25,Wed,0
1,300668,Annihilation,a biologist signs up for a dangerous secret e...,['Science Fiction'],2018-02-22,False,4232,6.3,0,0,...,1,0,0,0,0,116.0,2,22,Thu,0
2,299782,The Other Side of the Wind,surrounded by fans and skeptics grizzled dire...,"['Comedy', 'Drama']",2018-11-02,False,55,7.1,0,0,...,0,0,0,0,0,188.0,11,2,Fri,0
3,351044,Welcome to Marwen,when a devastating attack shatters mark hoganc...,"['Drama', 'Comedy', 'Fantasy']",2018-12-21,False,174,6.6,0,0,...,0,0,0,0,0,413.0,12,21,Fri,0
4,361292,Suspiria,a darkness swirls at the center of a world ren...,"['Thriller', 'Mystery', 'Horror', 'Fantasy']",2018-10-11,False,579,7.2,0,0,...,0,0,1,0,0,242.0,10,11,Thu,0


Recall the `plot_vote_avg_dist_by_word` function from the last notebook which plots overlaid histograms of ratings for movies that do and do not contain the word argument passed to it. We import that function from the `utils.py` file below and create an interactive widget that will plot the distribution for the word you enter into the `Distribution` box.

In [20]:
from utils import *

widgets.interact(
    lambda word: plot_vote_avg_dist_by_word(word),
    word = widgets.Text(
        value = "love",
        placeholder = "Type something",
        description = "Distribution: "
    )
);

interactive(children=(Text(value='love', description='Distribution: ', placeholder='Type something'), Output()…

For simplicity, we also choose some words that might make good features and provide a widget with a dropdown to select from those words to see their distributions.

In [21]:
potential_word_features = [
    "money", "america", "earn", "del", "quest", "treasure", "hunt", "kidnap", "must", "fear",
    "his", "her", "ring"
]

widgets.interact(
    lambda word: plot_vote_avg_dist_by_word(word),
    word = widgets.Dropdown(
        options = potential_word_features,
        value = "money",
        description = "Distribution: "
    )
);

interactive(children=(Dropdown(description='Distribution: ', options=('money', 'america', 'earn', 'del', 'ques…

Based on an examination of the distributions above, we choose 6 words below that might make good features and use `add_word_in_plot` to add these as columns to the `movies` dataframe, storing the result as `movies_with_words`.

In [22]:
good_word_features = ["money", "hunt", "treasure", "del", "fear", "ring"]

movies_with_words = add_word_in_plot(movies, good_word_features)
movies_with_words.head()

,id,title,overview,genres,release_date,adult,vote_count,vote_average,Action,Adventure,...,overview_len,month,day,day_of_week,money,hunt,treasure,del,fear,ring
0,299536,Avengers: Infinity War,as the avengers and their allies have continue...,"['Adventure', 'Action', 'Fantasy']",2018-04-25,False,12490,8.3,1,1,...,490.0,4,25,Wed,0,0,0,0,0,0
1,300668,Annihilation,a biologist signs up for a dangerous secret e...,['Science Fiction'],2018-02-22,False,4232,6.3,0,0,...,116.0,2,22,Thu,0,0,0,0,0,0
2,299782,The Other Side of the Wind,surrounded by fans and skeptics grizzled dire...,"['Comedy', 'Drama']",2018-11-02,False,55,7.1,0,0,...,188.0,11,2,Fri,0,0,0,0,0,0
3,351044,Welcome to Marwen,when a devastating attack shatters mark hoganc...,"['Drama', 'Comedy', 'Fantasy']",2018-12-21,False,174,6.6,0,0,...,413.0,12,21,Fri,0,0,0,0,0,0
4,361292,Suspiria,a darkness swirls at the center of a world ren...,"['Thriller', 'Mystery', 'Horror', 'Fantasy']",2018-10-11,False,579,7.2,0,0,...,242.0,10,11,Thu,0,0,0,0,0,0


### Testing with Word Features

We now use sklearn's `train_test_split` function to create a train-test split of `movies_with_words` and concatenate the list of possible features with the list `good_word_features` to rerun the `stochastic_choose_features` function below.

In [23]:
train, test = train_test_split(movies_with_words)

In [24]:
possible_features += good_word_features
stochastic_features = feature_pipeline(
    one_stochastic_iteration, 
    choose_random_features,
    reduce_iterations, 
    train, 
    possible_features
)
stochastic_features = ray.get(ray.get(stochastic_features))[0]
print("Features chosen: ", stochastic_features)
print("Features KFold CV RMSE: ", run_kfold(train, stochastic_features))

Features chosen:  ['Western', 'TV Movie', 'Action', 'Thriller', 'Family', 'Documentary', 'wkdy_Sat', 'Romance', 'Animation', 'day', 'History', 'Crime', 'wkdy_Tue', 'Music', 'War', 'money', 'Science Fiction', 'Drama', 'Fantasy', 'Comedy', 'vote_count', 'Horror', 'adult']
Features KFold CV RMSE:  2.0297106088066164


To test our newly chosen features, we process the `train` and `test` dataframes and run the predictor with `run_lin_reg`.

In [25]:
X_train, y_train = process_data(train, stochastic_features)
X_test, y_test = process_data(test, stochastic_features)

run_lin_reg(X_train, y_train, X_test, y_test)

Training RMSE: 2.010
Testing RMSE: 1.993


1.9925003177477458

## L2 Regularization

For the last portion of this notebook, we want to see if using Ridge Regression (L2 Regularization) will help improve the accuracy of the predictor. We begin by using `lm.RidgeCV` in conjunction with the train and test sets we just created to check the chosen value of $\alpha$ and its RMSE.

In [28]:
kf = KFold(n_splits = 5)
ridge = lm.RidgeCV(alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], cv = kf, normalize=True)
ridge.fit(X_train, y_train)
print("alpha: {:.3f}".format(ridge.alpha_))

alpha: 0.100


In [29]:
rmse(y_train, ridge.predict(X_train))

2.011302981035286

### Hyperparameter Search

Because `sklearn.linear_model.RidgeCV` will choose the best hyperparameter $\alpha$ for us, we define a function below which uses a gradient descent algorithm and `RidgeCV` to choose the value of $\alpha$. The function `find_alpha` below uses the algorithm to find the optimal $\alpha$; it runs until the absolute difference between the two values of $\alpha$ being considered is less than `tol`.

In [34]:
def find_alpha(X_train, y_train, tol=1e-8, verbose=True):
    alpha1, alpha2 = 0.1, 1
    kf = KFold(n_splits = 5)
    while abs(alpha1 - alpha2) > tol:
        ridge = lm.RidgeCV(alphas = [alpha1, alpha2], cv = kf, normalize=True)
        ridge.fit(X_train, y_train)
        if ridge.alpha_ == alpha1:
            alpha2 = alpha1 + 0.5 * (alpha2 - alpha1)
        else:
            alpha1 = alpha2 + 0.5 * (alpha1 - alpha2)
    if verbose:
        print("alpha: {:.5f}".format(alpha1))
    return alpha1, rmse(y_train, ridge.predict(X_train))

In [35]:
find_alpha(X_train, y_train)

alpha: 0.14686


(0.14685733318328856, 2.012139794493347)

## Conclusions

Despite all of the feature engineering that we've done, we can't get an RMSE lower than around 2 (40% error). This doesn't bode well for the success of the predictor. In later analyses, I will attempt to bring in other factors that don't come by default from the TMDb API.